# Using K-Nearest Neighbors Method to Predict IMDb Ratings

## Data Loading
Data from Kaggle. There are three datasets on movie name, ratings, etc. for each movie. We combine
basic info of movies and their complete ratings together to predict.

In [1]:
# Import packages
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
from matplotlib import pyplot as plt
# Import datasets
movies = pd.read_csv('dataset/IMDb movies.csv')
ratings = pd.read_csv('dataset/IMDb ratings.csv')
names = pd.read_csv('dataset/IMDb names.csv')

/Users/shizhixu/opt/anaconda3/envs/csci5523/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


We now present samples of each datasets.

In [2]:
movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [3]:
ratings.head()

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,...,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0
1,tt0000574,6.1,589,6.3,6.0,57,18,58,137,139,...,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0
2,tt0001892,5.8,188,6.0,6.0,6,6,17,44,52,...,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0
3,tt0002101,5.2,446,5.3,5.0,15,8,16,62,98,...,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0
4,tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,...,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0


In [4]:
names.head()

,imdb_name_id,name,birth_name,height,bio,birth_details,date_of_birth,place_of_birth,death_details,date_of_death,place_of_death,reason_of_death,spouses_string,spouses,divorces,spouses_with_children,children
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177.0,"Fred Astaire was born in Omaha, Nebraska, to J...","May 10, 1899 in Omaha, Nebraska, USA",1899-05-10,"Omaha, Nebraska, USA","June 22, 1987 in Los Angeles, California, USA ...",1987-06-22,"Los Angeles, California, USA",pneumonia,Robyn Smith (27 June 1980 - 22 June 1987) (hi...,2,0,1,2
1,nm0000002,Lauren Bacall,Betty Joan Perske,174.0,Lauren Bacall was born Betty Joan Perske on Se...,"September 16, 1924 in The Bronx, New York City...",1924-09-16,"The Bronx, New York City, New York, USA","August 12, 2014 in New York City, New York, US...",2014-08-12,"New York City, New York, USA",stroke,Jason Robards (4 July 1961 - 10 September 196...,2,1,2,3
2,nm0000003,Brigitte Bardot,Brigitte Bardot,166.0,"Brigitte Bardot was born on September 28, 1934...","September 28, 1934 in Paris, France",1934-09-28,"Paris, France",NaN,NaN,NaN,NaN,Bernard d'Ormale (16 August 1992 - present)\n...,4,3,1,1
3,nm0000004,John Belushi,John Adam Belushi,170.0,"John Belushi was born in Chicago, Illinois, US...","January 24, 1949 in Chicago, Illinois, USA",1949-01-24,"Chicago, Illinois, USA","March 5, 1982 in Hollywood, Los Angeles, Calif...",1982-03-05,"Hollywood, Los Angeles, California, USA",acute cocaine and heroin intoxication,Judith Belushi-Pisano (31 December 1976 - 5 M...,1,0,0,0
4,nm0000005,Ingmar Bergman,Ernst Ingmar Bergman,179.0,"Ernst Ingmar Bergman was born July 14, 1918, t...","July 14, 1918 in Uppsala, Uppsala län, Sweden",1918-07-14,"Uppsala, Uppsala län, Sweden","July 30, 2007 in Fårö, Gotlands län, Sweden (...",2007-07-30,"Fårö, Gotlands län, Sweden",natural causes,Ingrid Bergman (11 November 1971 - 20 May 199...,5,4,5,8


## Data Analysis
We now merge movies and ratings datasets together for analysis, training and testing.
Firstly, combine them together and show results

In [5]:
# Merge!
merged = pd.merge(left=movies, right=ratings, left_on='imdb_title_id', right_on='imdb_title_id')
print(merged.shape)
merged.head()

(85855, 70)


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0


We now clear the items with missing values and show amount of items left

In [6]:
merged = merged.dropna(how='any',axis=0)
print(merged.shape)

(4866, 70)


In this step, we analyze data types of each column and see which should be dropped
and which should be converted.

In [7]:
pd.set_option('display.max_rows', 70)
print(merged.dtypes)
pd.reset_option('display.max_rows')

imdb_title_id                 object
title                         object
original_title                object
year                          object
date_published                object
genre                         object
duration                       int64
country                       object
language                      object
director                      object
writer                        object
production_company            object
actors                        object
description                   object
avg_vote                     float64
votes                          int64
budget                        object
usa_gross_income              object
worlwide_gross_income         object
metascore                    float64
reviews_from_users           float64
reviews_from_critics         float64
weighted_average_vote        float64
total_votes                    int64
mean_vote                    float64
median_vote                  float64
votes_10                       int64
v

## Data Preprocessing

Based on data above, we drop columns related to titles, description and publishing date since they have no contribution
to our training and prediction. For publishing date, since we are more focusing on the year rather than the exact day
of publication, it is dropped as well.

In [8]:
# MergedMINI
mergedmini = merged
mergedmini = mergedmini.drop(['imdb_title_id', 'title', 'original_title', 'description', 'date_published'], axis=1)

We now convert some data to int/ float types and other preprocessing steps. Here we transform
production year to int and genre, country, language, directors, production company and writers to
categorical data, then use digits to represent them for better training and testing results.

In [9]:
# Remove movies published before 1970
mergedmini = mergedmini.astype({'year' : 'int'})
mergedmini = mergedmini[mergedmini['year'] > 1970]
# Converting objects to discrete values Country, Language
# Country first, then language
mergedmini['country'] = mergedmini['country'].astype("category")
mergedmini['language'] = mergedmini['language'].astype("category")
mergedmini['country_code'] = mergedmini['country'].cat.codes
mergedmini['lang_code'] = mergedmini['language'].cat.codes
# Production Company
mergedmini['production_company'] = mergedmini['production_company'].astype("category")
mergedmini['company_code'] = mergedmini['production_company'].cat.codes
# Directors
mergedmini['director'] = mergedmini['director'].astype("category")
mergedmini['director_code'] = mergedmini['director'].cat.codes
# Writers
mergedmini['writer'] = mergedmini['writer'].astype("category")
mergedmini['writer_code'] = mergedmini['writer'].cat.codes
# Genres
mergedmini['genre'] = mergedmini['genre'].astype("category")
mergedmini['genre_code'] = mergedmini['genre'].cat.codes
# Actors
mergedmini['actors'] = mergedmini['actors'].astype("category")
mergedmini['actors_code'] = mergedmini['actors'].cat.codes
# Cleaning original attributes
mergedmini = mergedmini.drop(['country', 'language', 'director', 'production_company', 'writer', 'genre', 'actors'], axis=1)

Now we deal with budgets and incomes. All currency will be converted to USD.

In [10]:
from currency_converter import CurrencyConverter
import re

# Grab only the currency attributes from the data.
budget = mergedmini['budget']
usa_gross_income = mergedmini['usa_gross_income']
worldwide_gross_income = mergedmini['worlwide_gross_income']

'''Use regular expressions to split the string into the currency code
and the value.'''
def split_currency_string(string):
    """Function takes a currency string and parses the currency code
    and currency value and stores it in a tuple.
    """

    match = re.match(r"(\$|[a-z]+) (\d+)", string, re.I)

    if match:
        items = match.groups()
        return items

c = CurrencyConverter()

'''Some of the currency codes in the data set don't match up with the converter.
We define a dictionary that contains those mis-labeled currency codes and
map them to the correct currency code for the converter.
'''
currency_codes = {
'$': 'USD',
'RUR': 'RUB',
}

'''There are a few currencies in the data set that are obsolete. These are currencies that were
used prior to the introduction of the Euro. This hash table maps each of the obsolete European
currencies to their conversion rate to the Euro.
'''
obsolete_euro_currencies = {
    'DEM': 0.511364,
    'FRF': 0.152449,
    'ITL': 0.000516457,
    'BEF': 0.0247894,
    'ESP': 0.00601012,
    'ATS': 0.0726728,
    'FIM': 0.168188
}

'''There are currencies in the data set that are not included in the currency converter. To
bypass this problem, we map each of these currencies to their conversion rate to the Euro.
'''
other_currencies = {
    'CLP': 0.0012,
    'NGN': 0.0022,
    'ARS': 0.0090,
    'DOP': 0.015
}

def convert_currency_to_USD(currency_code: str, currency_value: str):
    """The function takes in two arguments, a currency code and a currency value,
    and converts them to USD.
    """

    if currency_code in currency_codes:
        new_currency = (currency_codes[currency_code], currency_value)
        converted_value = c.convert(int(new_currency[1]), new_currency[0], 'USD')
    elif currency_code in obsolete_euro_currencies:
        obsolete_currency = int(currency_value)                             # Grab the value of the obsolete currency.
        euro = obsolete_currency * obsolete_euro_currencies[currency_code]  # Convert the deprecated currency value to the EUR.
        converted_value = c.convert(euro, 'EUR', 'USD')                     # Convert EUR to USD.
    elif currency_code in other_currencies:
        other_currency = int(currency_value)
        euro = other_currency * other_currencies[currency_code]
        converted_value = c.convert(euro, 'EUR', 'USD')
    else:
        converted_value = c.convert(int(currency_value), currency_code, 'USD')  # Convert all currencies to USD.

    return converted_value

'''Since we are attempting to write back to the original data frame, we need to
ensure that we disable the chained assignment protection in the Pandas options.
'''
pd.options.mode.chained_assignment = None  # default='warn'

def currency_str_to_float(df, column_name):
    """Takes in a data frame and a column name. Converts each row into USD and
    modifies the data frame in place.
    """

    for i in range(0, len(df)):
        currency = split_currency_string(df[column_name].iloc[i])       # Split the string into a code and value.
        converted = convert_currency_to_USD(currency[0], currency[1])   # Convert the currency value to USD.
        df[column_name].iloc[i] = converted                             # Apply the converted value to the data frame.

currency_str_to_float(mergedmini, 'budget')                      # Convert Budget currencies to USD.
currency_str_to_float(mergedmini, 'usa_gross_income')            # Convert USA Gross Income to USD.
currency_str_to_float(mergedmini, 'worlwide_gross_income')       # Convert Worldwide Gross Income to USD.


## Training and Prediction
In this step, we try to use the dataset processed for training and prediction. We will separate the dataset with 70% for
training and 30% for testing. We will set 10 bins for the final ratings from 0 to 10 with interval 1.

In [11]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
mergedmini['avg_vote_group'] = pd.cut(mergedmini['weighted_average_vote'], bins=10, labels=labels)
y = mergedmini['avg_vote_group']
mergedmini = mergedmini.drop(['weighted_average_vote'], axis=1)
X_train, X_holdout, y_train, y_holdout = train_test_split(mergedmini.values, y, test_size=0.3,
random_state=17)

knn = KNeighborsClassifier(n_neighbors=5)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [12]:
from sklearn.metrics import accuracy_score
# from sklearn.metrics import confusion_matrix
knn_pred = knn.predict(X_holdout_scaled)
print (f" Accuracy for k-nn is {accuracy_score(y_holdout, knn_pred)}")

 Accuracy for k-nn is 0.8806699232379623
